# Average and deviation std of the models for the loss
The loss is the Mean Square Error, basicaly same stuff of Euclidean distance between two matrix

In [1]:
import os
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def extract_lastLoss(directory):
    # Extract last loss from all the models
    lastLoss_tuples = []
    # Loop through each file in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)       # Construct the full file path
        if os.path.isfile(file_path):       # Check if it is a file
            result_sim = np.load(file_path, allow_pickle=True)
            lastLoss_tuples.extend([(label, array[-1]) for label, array in result_sim if 'loss' in label])

    # Creating df
    return pd.DataFrame(lastLoss_tuples, columns=['label', 'value'])


In [3]:
def models_repmin_mean_devstd(df, name_models):
    # Extracting the base label (e.g., 'LabelA', 'LabelB')
    df['label_no_rep'] = df['label'].apply(lambda x: x.split('_rep')[0])
    # Group with the base label and find min
    min_lastLoss_s = df.groupby('label_no_rep')['value'].min()
    # print(min_lastLoss_s)

    mean_models= []
    std_dev_models =[]
    for name in name_models:
        # take the same model 
        model_s = min_lastLoss_s[min_lastLoss_s.index.str.contains(name)]
        mean_models.append(model_s.mean())
        std_dev_models.append(model_s.std())
    
    return mean_models, std_dev_models

In [4]:
def models_repmax_mean_devstd(df, name_models):
    # Extracting the base label (e.g., 'LabelA', 'LabelB')
    df['label_no_rep'] = df['label'].apply(lambda x: x.split('_rep')[0])
    # Group with the base label and find min
    max_lastLoss_s = df.groupby('label_no_rep')['value'].max()
    # print(min_lastLoss_s)

    mean_models= []
    std_dev_models =[]
    for name in name_models:
        # take the same model 
        model_s = max_lastLoss_s[max_lastLoss_s.index.str.contains(name)]
        mean_models.append(model_s.mean())
        std_dev_models.append(model_s.std())
    
    return mean_models, std_dev_models

In [1]:
import torch
import tqdm

In [4]:
# Example: 6x6 full matrix and three 2x2 blocks
N = 2000
blocks = [torch.randn(2, 2) for _ in range(N // 2)]
full_matrix = torch.randn(N, N)

# Prepare batches
block_tensor = torch.stack(blocks)  # Shape: [3, 2, 2]
# Extract corresponding 2-row segments from the full matrix
segment_tensor = torch.stack([full_matrix[2*i:2*i+2] for i in range(N // 2)])  # Shape: [3, 2, N]

# Batch matrix multiplication
for _ in tqdm.trange(1000):
    result = torch.bmm(block_tensor, segment_tensor)  # Result shape: [3, 2, N]

    # Assemble the result back into the full result matrix if needed
    result_matrix = torch.zeros_like(full_matrix)
    for i in range(result.shape[0]):
        result_matrix[2*i:2*i+2, :] = result[i]

100%|██████████| 1000/1000 [00:14<00:00, 66.88it/s]


In [5]:

# Example blocks and full matrix
blocks = [torch.randn(2, 2) for _ in range(N // 2)]  # List of 2x2 blocks
full_matrix = torch.randn(N, N)

# Result matrix initialized to zero
result = torch.zeros_like(full_matrix)

for _ in tqdm.trange(1000):
    # Apply each block to the corresponding submatrix of the full matrix
    for i, block in enumerate(blocks):
        start_index = 2 * i
        result[start_index:start_index+2, :] = block @ full_matrix[start_index:start_index+2, :]


100%|██████████| 1000/1000 [00:11<00:00, 87.43it/s]


In [39]:
# Define the size of your matrices
n = 4  # for a 4x4 matrix

# Create a full matrix
torch.manual_seed(37)
full_matrix = 1j*torch.randn(n, n)

# Create a sparse matrix with elements only at specified indices
sparse_matrix = 1j*torch.zeros(n, n)
# Assign values to specified indices
sparse_matrix[0, 0] = 1j*1.5  # example value
sparse_matrix[1, 0] = 1j*2.3  # example value
sparse_matrix[2, 2] = 1j*-0.6 # example value
sparse_matrix[3, 2] = 1j*0.8  # example value

sparse_matrix[0, 1] = 1j*1.5*1.65  # example value
sparse_matrix[1, 1] = 1j*2.3*1.65  # example value
sparse_matrix[2, 3] = 1j*-0.6*1.65 # example value
sparse_matrix[3, 3] = 1j*0.8*1.65  # example value

# Perform element-wise multiplication
result = sparse_matrix @ full_matrix

print(result)

tensor([[ 2.4425+0.j, -2.1900+0.j, -4.9393+0.j, -0.9951+0.j],
        [ 3.7451+0.j, -3.3581+0.j, -7.5736+0.j, -1.5257+0.j],
        [-0.9441+0.j, -1.5235+0.j, -1.2227+0.j,  1.0526+0.j],
        [ 1.2589+0.j,  2.0313+0.j,  1.6302+0.j, -1.4035+0.j]])


In [45]:
diag_elements1 = torch.tensor([1j*1.5, 1j*2.3, 1j*-0.6, 1j*0.8])
diag_elements2 = torch.tensor([1j*1.5, 1j*2.3, 1j*-0.6, 1j*0.8])*1.65


# Create a new matrix by copying even rows into odd rows
even_indices_values = full_matrix[0::2]
new_matrix1 = full_matrix.clone()  # Clone the original matrix to preserve it
# new_matrix1[1::2] = new_matrix1[0::2]  # Copy even rows into odd rows
new_matrix1[1::2] = even_indices_values

result1 = diag_elements1.view(-1, 1) * new_matrix1

new_matrix2 = full_matrix.clone()  # Clone the original matrix to preserve it
new_matrix2[:-1:2] = new_matrix2[1::2]

result2 = diag_elements2.view(-1, 1) * new_matrix2

result = result1 + result2

result


tensor([[ 2.4425+0.j, -2.1900+0.j, -4.9393+0.j, -0.9951+0.j],
        [ 3.7451+0.j, -3.3581+0.j, -7.5736+0.j, -1.5257+0.j],
        [-0.9441+0.j, -1.5235+0.j, -1.2227+0.j,  1.0526+0.j],
        [ 1.2589+0.j,  2.0313+0.j,  1.6302+0.j, -1.4035+0.j]])

In [50]:
import numpy as np
import torch
import torch.nn as nn

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"


class mlayer_MMIs_odd(nn.Module):
    r""" Create odd MMIs matrix
    0______0

    1__  __1
       \/
    2__/\__2
    
    3______3
    """
    def __init__(self, N: int,
                 insertion_loss_MMI: torch.Tensor,          # Dimension should be N//2
                 imbalance_MMI: torch.Tensor):
        
        if N%2 == 1: Exception('N is odd!!! NONONO, put it even!!!')
        self.N = N
        
        attenuation = torch.sqrt(1-insertion_loss_MMI)          # They are in vector format
        MMI11 = attenuation*torch.sqrt(1/2+imbalance_MMI)
        MMI12 = attenuation*1.j*torch.sqrt(1/2-imbalance_MMI)
        MMI21 = attenuation*1.j*torch.sqrt(1/2-imbalance_MMI)
        MMI22 = attenuation*torch.sqrt(1/2+imbalance_MMI)
        
        self._diag1 = torch.zeros(N, dtype=torch.cfloat)
        self._diag2 = torch.zeros(N, dtype=torch.cfloat)
        self._diag1[0] = 1
        self._diag1[-1] = 1
        for i in range(1, N-1, 2):
            print(i)
            self._diag1[i] = MMI11[i//2]
            self._diag1[i+1] = MMI12[i//2]
            self._diag2[i] = MMI21[i//2]
            self._diag2[i+1] = MMI22[i//2]

N = 6
insertion_loss_MMI = torch.Tensor([0 for _ in range(N//2)])
imbalance_MMI = torch.Tensor([0 for _ in range(N//2)])

cioa = mlayer_MMIs_odd(N, insertion_loss_MMI, imbalance_MMI)

1
3


In [6]:
import torch

# Define the size of the original square matrix
N = 6

# Create a sample NxN matrix
original_matrix = torch.randn(N, N)

# Reshape the original matrix into the desired shape
reshaped_matrix = original_matrix.view(N // 2, 2, N)

print("Original Matrix Shape:", original_matrix.shape)
print("Reshaped Matrix Shape:", reshaped_matrix.shape)
print(original_matrix)


orig = reshaped_matrix.view(original_matrix.shape[0], -1)
original_matrix.shape[0]

Original Matrix Shape: torch.Size([6, 6])
Reshaped Matrix Shape: torch.Size([3, 2, 6])
tensor([[ 0.7838, -0.7593,  0.6416,  0.3753,  0.8976, -0.5454],
        [ 1.2951,  0.0497,  2.5351, -0.1539,  0.1379,  1.3129],
        [-0.5484, -0.5114, -1.0520, -0.0981,  0.2616,  0.4543],
        [ 0.7163,  1.2418,  1.6048,  0.1609,  0.9362,  0.1414],
        [ 1.8085, -0.1083,  1.9048, -0.0193, -0.2414, -2.1111],
        [ 1.7789,  0.0567, -0.1200,  0.7928, -1.2071,  0.0577]])


6

In [6]:
import torch

# Creating a 2D array
mmi_i_losses_matrix = torch.Tensor([[1, 2, 3],
                                [4, 5, 6],
                                [7, 8, 9]])

# Select the 2nd column (remember, Python uses zero-based indexing)
selected_column = mmi_i_losses_matrix[:, 1]

print(selected_column)

tensor([2., 5., 8.])


In [30]:
import torch

# Assume _n_mmi, atten, tau, and kappa are defined as:
_n_mmi = 10  # example size
atten = torch.tensor([0.5, 0.6, 0.7, 0.8])  # example attenuation values
tau = torch.tensor([4.0, 3.0, 2.0, 1.0])    # example tau values
kappa = torch.tensor([1.0, 1.1, 1.2, 1.3])

# Calculate the components
# Create the elements for the matrix
a = atten + 0.j
b = tau + 0.j

conn = torch.zeros(4)
conn[0] = 1


mmi_elements = torch.stack([
    torch.stack([a, b], dim=-1),
    torch.stack([b, a], dim=-1)
], dim=-2)

# mmi_elements = torch.stack([[a, b],
#                             [b, a]], dim=-2)


stak1 = torch.stack([a, b], dim=-1),
# Expand to _n_mmi batches and reshape
# _mmis = mmi_elements.expand(_n_mmi, 2, 2)

# Output the _mmis tensor
print(a)
print(b)
print(mmi_elements)

tensor([0.5000+0.j, 0.6000+0.j, 0.7000+0.j, 0.8000+0.j])
tensor([4.+0.j, 3.+0.j, 2.+0.j, 1.+0.j])
tensor([[[0.5000+0.j, 4.0000+0.j],
         [4.0000+0.j, 0.5000+0.j]],

        [[0.6000+0.j, 3.0000+0.j],
         [3.0000+0.j, 0.6000+0.j]],

        [[0.7000+0.j, 2.0000+0.j],
         [2.0000+0.j, 0.7000+0.j]],

        [[0.8000+0.j, 1.0000+0.j],
         [1.0000+0.j, 0.8000+0.j]]])


In [32]:
import torch

# Example of dynamically creating a list of matrices
matrices = []
for i in range(1, 5):  # Dynamically deciding the number of matrices
    matrices.append(torch.tensor([[i, i+1], [i+2, i+3]]))

conn = torch.zeros(4)
conn = torch.Tensor([1])
# Use * to unpack the list of tensors into the function
block_matrix = torch.block_diag(conn, *mmi_elements)

print(mmi_elements)
print(block_matrix)

TypeError: expected Tensor as element 0 in argument 0, but got list

In [34]:
condition = torch.tensor([True, False, True, False])

# Tensors from which to choose values
x = torch.tensor([1, 2, 3, 4])
y = torch.tensor([100, 200, 300, 400])

# Using torch.where
result = torch.where(condition, x, y)
result

tensor([  1, 200,   3, 400])

In [40]:
import torch

# Tensors x and y, each with 2 elements
x = torch.tensor([1, 2])
y = torch.tensor([100, 200])

# Condition tensor, tailored to match a repeated scenario
condition = torch.tensor([True, False])

# Repeat x, y, and condition to make the length 4
x_repeated = x.repeat(2)  # Repeats the entire tensor
y_repeated = y.repeat(2)
condition_repeated = condition.repeat(2)  # [True, False, True, False]

# Use torch.where to select values based on the condition
result = torch.where(condition_repeated, x_repeated, y_repeated)

print(x_repeated)
print(result)


tensor([1, 2, 1, 2])
tensor([  1, 200,   1, 200])
